### MCP(Model Context Protocol) 模型上下文协议

#### 为什么需要 MCP、Function Calling？ 
- LLM 数据库内容有限
    - LLM 只知道其训练数据中包含的信息
    - 训练 LLM 需要庞大的计算资源和时间
    - 问题：LLM 所了解的知识总是过时的；幻觉严重
- LLM 缺乏专业知识
    - LLM 一般使用公开数据进行训练
    - 无法理解特定业务场景中的专业知识和信息

#### 需求
能否让 LLM 获取到数据库外的更多信息？

#### Tool Use
- The agent learns to call external APIs for extra information that is missing from the model weights (often hard to change after pre-training), including current information, code execution capability, access to proprietary information sources and more.
- 智能体学习调用外部 API 以获取模型权重中缺失的额外信息（通常在预训练后难以更改），包括当前信息、代码执行能力、对专有信息源的访问等。

#### 提示词工程
通过提示词工程，定义协议，可以让用户按照协议输入，也可以让大模型返回协议，解析文本中的返回，之后**交给工程代码去调用**。

##### 流程
<img src=pics/prompts-engine-timeline.png width=50%>

#### 示例

In [ ]:
/*
你是一个智能助手，具有以下工具：

1. 天气工具 (WEATHER)
   - 格式：WEATHER: 城市名称
   - 功能：获取指定城市的当前天气情况

2. 计算工具 (CALCULATE)
   - 格式：CALCULATE: 数学表达式
   - 功能：执行数学计算

重要规则：
- 必须严格遵循上述格式
- 只有在确实需要使用工具时才使用
- 工具调用应该是输出的唯一内容
- 使用工具后，还需要对结果进行自然语言解释

示例：

用户: 我想知道北京的天气
助手: WEATHER: 北京

用户: 帮我计算15乘以23
助手: CALCULATE: 15 * 23
*/

! \
export API_KEY=sk-fpNfbaWHy4jMVkjg6e0725E735A24768A46e722e0b0115Ad; \
echo "API_KEY=$API_KEY"; \
export PROMPTS="你是一个智能助手，具有以下工具：\n\n1. 天气工具 (WEATHER)\n   - 格式：WEATHER: 城市名称\n   - 功能：获取指定城市的当前天气情况\n\n2. 计算工具 (CALCULATE)\n   - 格式：CALCULATE: 数学表达式\n   - 功能：执行数学计算\n\n重要规则：\n- 必须严格遵循上述格式\n- 只有在确实需要使用工具时才使用\n- 工具调用应该是输出的唯一内容\n- 使用工具后，还需要对结果进行自然语言解释\n\n示例：\n\n用户: 我想知道北京的天气\n助手: WEATHER: 北京\n\n用户: 帮我计算15乘以23\n助手: CALCULATE: 15 * 23"; \
echo "PROMPTS=$PROMPTS"; \
curl -s "https://api.gptapi.us/v1/chat/completions" \
    -H "Content-Type: application/json" \
    -H "Authorization: Bearer $API_KEY" \
    -d '{  \
        "model": "gpt-4o-mini",     \
        "messages": [   \
            {   \
                "role": "system",    \
                "content": "'"$PROMPTS"'" \
            },  \
    	    {   \
    	        "role": "user", \
    	        "content": "我想知道北京的天气"   \
    	    },  \
            {   \
                "role": "assistant", \
                "content": "WEATHER: 北京\n" \
            },  \
            {   \
                "role": "user", \
                "content": "北京的天气为晴天"   \
            } \
        ] \
    }' | jq

##### 痛点
- 早期大模型能力不足，难以严格遵守指令
- 早期大模型返回内容可能会不标准，可能会多余返回：“好的，下面是您需要的..."类似的开场词
- 早期大模型上下文有限，难以处理复杂参数工具

#### Function Calling

>- You can give the model access to your own custom code through function calling. Based on the system prompt and messages, the model may decide to call these functions — instead of (or in addition to) generating text or audio.
> You'll then execute the function code, send back the results, and the model will incorporate them into its final response.
>- 你可以通过函数调用让模型访问你自己的自定义代码。根据系统提示和消息，模型可能会决定调用这些函数——而不是（或除了）生成文本或音频。
> 然后，你将执行函数代码，发回结果，模型会将这些结果整合到其最终响应中。

Function Calling 是大型语言模型（LLM）的一项核心功能，它允许模型在生成文本响应的同时，智能地识别何时需要调用外部工具或 API 来获取额外信息或执行特定操作。

##### 步骤
1. 用户提出请求
2. 模型识别意图
3. 模型生成函数调用参数
4. 外部代码执行函数
5. 结果返回给模型
6. 模型生成最终响应

<img src="pics/function-calling-diagram-steps.png" width="40%" style="text-align">

##### OpenAI Function Calling

In [62]:
!   \
export API_KEY=sk-fpNfbaWHy4jMVkjg6e0725E735A24768A46e722e0b0115Ad; \
curl -s "https://api.gptapi.us/v1/chat/completions" \
    -H "Content-Type: application/json" \
    -H "Authorization: Bearer $API_KEY" \
    -d '{\
        "model": "gpt-4o-mini", \
        "messages": [\
            { \
                "role": "user", \
                "content": "现在北京的天气是什么?" \
            } \
        ], \
        "tools": [\
            { \
                "type": "function", \
                "function": { \
                    "name": "get_current_weather", \
                    "description": "获取一个位置的当前天气", \
                    "parameters": { \
                        "type": "object", \
                        "properties": { \
                            "location": { "type": "string", "description": "一个位置, 例如: 北京, 上海, 广州" }, \
                            "unit": { "type": "string", "enum": ["celsius", "fahrenheit"] } \
                        }, \
                        "required": ["location"] \
                    } \
                } \
            } \
        ], \
        "tool_choice": "auto" \
    }' | jq

{
  "id": "chatcmpl-BXTI9B8RdjuZ4uzUdbVPZE1Ih3QhR",
  "object": "chat.completion",
  "created": 1747317121,
  "model": "gpt-4o-mini-2024-07-18",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "tool_calls": [
          {
            "function": {
              "arguments": "{\"location\":\"北京\"}",
              "name": "get_current_weather",
              "parameters": null
            },
            "id": "call_hZBbdshnxI66eoqBCP4rRGjS",
            "type": "function"
          }
        ]
      },
      "finish_reason": "tool_calls"
    }
  ],
  "usage": {
    "prompt_tokens": 76,
    "completion_tokens": 16,
    "total_tokens": 92,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  },
  "system_fingerprint": "fp_7a53abb7a2"
}


In [ ]:
!\
export API_KEY=sk-fpNfbaWHy4jMVkjg6e0725E735A24768A46e722e0b0115Ad; \
curl -s "https://api.gptapi.us/v1/chat/completions" \
    -H "Content-Type: application/json" \
    -H "Authorization: Bearer $API_KEY" \
    -d '{   \
        "model": "gpt-4o-mini", \
        "messages": [ \
            { \
                "role": "user", \
                "content": "北京天气怎么样？" \
            }, \
            { \
                "role": "assistant",   \
                "tool_calls": [ \
                    { \
                        "id": "call_hZBbdshnxI66eoqBCP4rRGjS", \
                        "type": "function", \
                        "function": { \
                            "name": "get_current_weather", \
                            "arguments": "{\"location\":\"北京\"}" \
                        } \
                    } \
                ] \
            }, \
            { \
                "role": "tool", \
                "tool_call_id": "call_hZBbdshnxI66eoqBCP4rRGjS", \
                "content": "{\"temperature\": \"25\", \"description\": \"阴天\"}" \
            } \
        ], \
        "tools": [\
            { \
                "type": "function", \
                "function": { \
                    "name": "get_current_weather", \
                    "description": "获取一个位置的当前天气", \
                    "parameters": { \
                        "type": "object", \
                        "properties": { \
                            "location": { "type": "string", "description": "一个位置, 例如: 北京, 上海, 广州" }, \
                            "unit": { "type": "string", "enum": ["celsius", "fahrenheit"] } \
                        }, \
                        "required": ["location"] \
                    } \
                } \
            } \
        ], \
        "tool_choice": "auto" \
    }' | jq

#### Function Callings 比较

##### Function Define
- OpenAI <br>
    ```json
    {
      "model": "gpt-4o-mini",
      "messages": [{"role": "user", "content": "What's the weather like in Boston?"}],
      "tools": [
        {
          "type": "function",
          "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
              "type": "object",
              "properties": {
                "location": {"type": "string", "description": "The city and state"},
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
              },
              "required": ["location"]
            }
          }
        }
      ]
    }
    ```
- Gemini <br>
    ``` json
    {
      "contents": [{"role": "user", "parts": [{"text": "What's the weather like in Boston?"}]}],
      "tools": [
        {
          "functionDeclarations": [
            {
              "name": "get_current_weather",
              "description": "Get the current weather in a given location",
              "parameters": {
                "type": "object", 
                "properties": {
                  "location": {"type": "string"},
                  "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
                },
                "required": ["location"]
              }
            }
          ]
        }
      ]
    }
    ```
- Claude <br>
    ```json
    {
      "model": "claude-3-7-sonnet-20250219",
      "max_tokens": 1024,
      "tools": [
        {
          "name": "get_weather",
          "description": "Get the current weather in a given location",
          "input_schema": {
            "type": "object",
            "properties": {
              "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA"
              }
            },
            "required": [
              "location"
            ]
          }
        }
      ],
      "messages": [
        {
          "role": "user",
          "content": "What is the weather like in San Francisco?"
        }
      ]
    }
    ```
    <br>
<img src="pics/standards.png" width=40%>

#### 痛点

- 每个LLM服务商都有不同的 Function Calling 调用方法
- Function Call 需要用户或开发者手动执行

#### MCP(Model Context Protocol)

>- MCP is an open protocol that standardizes how applications provide context to LLMs. Think of MCP like a USB-C port for AI applications. Just as USB-C provides a standardized way to connect your devices to various peripherals and accessories, MCP provides a standardized way to connect AI models to different data sources and tools.
>- MCP是一个开放协议,用于标准化应用程序如何为LLM提供上下文。将MCP视为用于AI应用的USB-C端口。正如USB-C提供了一种将设备连接到各种外围设备和配件的标准化方式,MCP提供了一种标准化的方式将AI模型连接到不同的数据源和工具。

<br>
<img src="pics/api_overview.png" width=60%>
<img src="pics/mcp_overview.png" width=60%>

#### 解决痛点

- 各LLM提供商的 Function Calling 调用标准不一致(尽管大部分提供商都做了OpenAI API格式兼容)
- Function Call 需要开发者或用户手动去执行

#### 额外好处

- 可提供远程或本地的数据源以及工具，供LLM使用
- 相比完全由用户执行的 Function Call，MCP能够保护数据源

#### MCP架构
<br>
<img src="pics/mcp-architectures.png" width=60%>
<br>
 
- MCP 主机: 想要通过 MCP 访问数据的 Claude Desktop、IDE 或 AI 工具等程序
- MCP 客户端: 与服务器保持 1:1 连接的协议客户端
- MCP 服务器: 轻量级程序,每个程序通过标准化的模型上下文协议暴露特定功能
- 本地数据源: MCP 服务器可以安全访问的计算机文件、数据库和服务
- 远程服务: 通过互联网提供的外部系统(例如,通过API),MCP服务器可以连接到

<br>

<img src="pics/mcp-timeline.png" width=60%>

##### 通信

>- MCP uses JSON-RPC 2.0 as its wire format. The transport layer is responsible for converting MCP protocol messages into JSON-RPC format for transmission and converting received JSON-RPC messages back into MCP protocol messages.
>- MCP 使用 JSON-RPC 2.0 作为其导线格式。传输层负责将MCP协议消息转换为JSON-RPC格式,以便传输和将收到的JSON-RPC消息转换回MCP协议消息。

##### 通信方式
- STDIO
- ~~HTTP+SSE~~
- Streamable HTTP

##### ？

1. STDIO -> socket ?
2. Websocket ?

##### Awesome MCP Server

- Browser Search - 搜索浏览器资源
- DB - 数据库检索，MySQL，Postgre，Elastic Search...
- 原型图转代码 - Figma MCP Server, Master MCP Server...
- 各开放平台、云服务 - 飞书MCP Server(机器人、表格); Promethus...

##### 能为我们做什么（如今）？

- 基础架构、运维等： 结合监控系统MCP Server（Promethus, ...）实现基于LLM的数据分析等
- 任务处理： 结合各开放平台，实现写API也能实现，但是有点麻烦的自动任务处理，如：
    - 让LLM结合飞书MCP Server喊人修BUG，涉及流程： LLM分析用户请求 -> 飞书MCP操作表格，新增缺陷管理记录 -> 飞书MCP操作BOT，私聊或群发新缺陷通知
    - ...
- 辅助开发： 如：
    - 结合MySQL或其他DB MCP Server，实现后端数据校验，记录中各字段数值是否符合预期，验证算法是否准确
    - ...
- ~~吹牛逼： 最新技术趋势，趁热跟上~~